# Read ASC files

## Import libraries

In [1]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio

## Read DENUE file

In [22]:
gdf = gpd.read_file('data/denue/denue_inegi_final.gpkg')
gdf.drop(columns=['cve_loc',"numero_ext","fecha_alta"], inplace=True)
gdf.head()

,localidad,ageb,manzana,nom_estab,codigo_act,nombre_act,P6A14NAE,SBASC,PSDSS,OVSDE,OVSEE,OVSAE,OVPT,OVSREF,OVSINT,OVSCEL,OVHAC,geometry
0,Colima,0727,031,FARMACIA INTERCAMBIABLE,464111,Farmacias sin minisúper,2.805611,25.378627,20.178042,0.000000,0.252062,0.068744,1.489459,4.147571,27.978918,6.645280,15.879927,POINT (633121.957 2128794.720)
1,Colima,0746,034,FARMACIA GUADALUPANA,464112,Farmacias con minisúper,1.923077,10.915493,20.382166,0.109529,0.219058,0.000000,0.109529,2.957284,18.181818,6.681271,8.433735,POINT (633791.822 2128170.288)
2,Colima,0712,006,FARMACIA NUEVA IMPERIAL,464111,Farmacias sin minisúper,3.418803,21.231918,19.435484,0.000000,0.041580,0.000000,0.332640,3.783784,23.991684,9.147609,15.634096,POINT (634823.711 2128769.626)
3,Colima,0750,003,FARMACIA MI BARRIO,464111,Farmacias sin minisúper,4.090909,24.190178,20.730634,0.846325,0.579065,0.668151,0.846325,4.276169,31.403118,8.685969,14.610245,POINT (633641.784 2128038.406)
4,Colima,0750,010,FARMACIA LA PROVIDENCIA,464111,Farmacias sin minisúper,4.090909,24.190178,20.730634,0.846325,0.579065,0.668151,0.846325,4.276169,31.403118,8.685969,14.610245,POINT (633457.022 2127859.270)


## Ubicación de los archivos ASC

In [3]:
asc_files = glob.glob('data/Rasters/*/*.asc')
asc_files[:5]

['data/Rasters\\Hydrology\\Rain____0.0.asc',
 'data/Rasters\\Hydrology\\Rain____10200.0.asc',
 'data/Rasters\\Hydrology\\Rain____10500.0.asc',
 'data/Rasters\\Hydrology\\Rain____10800.0.asc',
 'data/Rasters\\Hydrology\\Rain____11100.0.asc']

# Read SINGLE ASC file

In [4]:
variable = asc_files[0].split('\\')[2].split('_')[0]
variable

'Rain'

In [5]:
step = asc_files[0].split('\\')[2].split('_')[-1][:-4]
step

'0.0'

In [6]:
grid = np.loadtxt(asc_files[0], skiprows=6)
null_value = -9999.
grid[grid == null_value] = np.nan
grid.shape

(1923, 1540)

In [7]:
def read_meta(file_src):
  meta = {}
  with open(file_src, 'r') as f:
    for i in range(6):
      line = f.readline()
      
      # remove empty spaces to keep only one
      line = ' '.join(line.split())
      
      # split line into key and value
      key, value = line.split(' ')
      
      # add key and value to dictionary
      meta[key] = value
  return meta

# Read MULTIPLE ASC files

In [8]:
def read_asc(file_src):
  variable = file_src.split('\\')[2].split('_')[0]
  step = file_src.split('\\')[2].split('_')[-1][:-4]
  
  meta = read_meta(file_src)
  
  grid = np.loadtxt(file_src, skiprows=6)
  null_value = -9999.
  grid[grid == null_value] = np.nan
  
  return grid, meta, variable, step
  
  
read_asc(asc_files[0])

(array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 {'NCOLS': '1540',
  'NROWS': '1923',
  'XLLCENTER': '631153.500000',
  'YLLCENTER': '2125184.250000',
  'CELLSIZE': '5.000',
  'NODATA_VALUE': '-9999.0'},
 'Rain',
 '0.0')

In [9]:
first_grid, first_meta, first_variable, first_step = read_asc(asc_files[0])


In [10]:
first_grid

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [11]:
gdf.head()

,localidad,ageb,manzana,nom_estab,codigo_act,nombre_act,geometry
0,Colima,0727,031,FARMACIA INTERCAMBIABLE,464111,Farmacias sin minisúper,POINT (633121.957 2128794.720)
1,Colima,0746,034,FARMACIA GUADALUPANA,464112,Farmacias con minisúper,POINT (633791.822 2128170.288)
2,Colima,0712,006,FARMACIA NUEVA IMPERIAL,464111,Farmacias sin minisúper,POINT (634823.711 2128769.626)
3,Colima,0750,003,FARMACIA MI BARRIO,464111,Farmacias sin minisúper,POINT (633641.784 2128038.406)
4,Colima,0750,010,FARMACIA LA PROVIDENCIA,464111,Farmacias sin minisúper,POINT (633457.022 2127859.270)


In [12]:
(633121.957 - 631153.5)/5

393.6914000000106

In [31]:
def work_progress(file_src, gdf, df_final):
  grid, meta, variable, step = read_asc(file_src)
  
  for i, row in gdf.iterrows():
    
    x_asc = int((row.geometry.centroid.x - float(meta['XLLCENTER']))/float(meta['CELLSIZE']))
    y_asc = int((row.geometry.centroid.y - float(meta['YLLCENTER']))/float(meta['CELLSIZE']))
    
    v_asc = grid[y_asc, x_asc]
    
    # nom_estab, nombre_act, x_asc, y_asc, v_asc, variable, step
    #row_df = [[row.nom_estab, row.nombre_act, row.geometry.centroid.x, row.geometry.centroid.y, v_asc, variable, step]]
    #row_df = pd.DataFrame(row_df, columns=['nom_estab', 'nombre_act', 'x', 'y', 'v_asc', 'variable', 'step'])
    
    row["v_asc"] = v_asc
    row["variable"] = variable
    row["step"] = step
    
    row_df = pd.DataFrame([row])
    
    
    df_final = pd.concat([df_final, row_df], ignore_index=True)

  return df_final
  
f = work_progress(asc_files[0], gdf, None)
f

,localidad,ageb,manzana,nom_estab,codigo_act,nombre_act,P6A14NAE,SBASC,PSDSS,OVSDE,...,OVSAE,OVPT,OVSREF,OVSINT,OVSCEL,OVHAC,geometry,v_asc,variable,step
0,Colima,0727,031,FARMACIA INTERCAMBIABLE,464111,Farmacias sin minisúper,2.805611,25.378627,20.178042,0.000000,...,0.068744,1.489459,4.147571,27.978918,6.645280,15.879927,POINT (633121.9573959348 2128794.7203419697),NaN,Rain,0.0
1,Colima,0746,034,FARMACIA GUADALUPANA,464112,Farmacias con minisúper,1.923077,10.915493,20.382166,0.109529,...,0.000000,0.109529,2.957284,18.181818,6.681271,8.433735,POINT (633791.8220921274 2128170.2879505726),0.0,Rain,0.0
2,Colima,0712,006,FARMACIA NUEVA IMPERIAL,464111,Farmacias sin minisúper,3.418803,21.231918,19.435484,0.000000,...,0.000000,0.332640,3.783784,23.991684,9.147609,15.634096,POINT (634823.7113330765 2128769.6261537718),0.0,Rain,0.0
3,Colima,0750,003,FARMACIA MI BARRIO,464111,Farmacias sin minisúper,4.090909,24.190178,20.730634,0.846325,...,0.668151,0.846325,4.276169,31.403118,8.685969,14.610245,POINT (633641.7836969274 2128038.4058942627),NaN,Rain,0.0
4,Colima,0750,010,FARMACIA LA PROVIDENCIA,464111,Farmacias sin minisúper,4.090909,24.190178,20.730634,0.846325,...,0.668151,0.846325,4.276169,31.403118,8.685969,14.610245,POINT (633457.0221438431 2127859.2699528662),NaN,Rain,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Colima,0731,026,UNIVERSIDAD MULTITECNICA PROFESIONAL,611311,Escuelas de educación superior del sector privado,4.306220,15.492958,17.206133,0.255537,...,0.511073,0.724020,1.618399,15.459966,5.749574,5.749574,POINT (632827.9149384073 2128394.2765015755),NaN,Rain,0.0
181,Colima,1265,020,UNIVERSIDAD INETEP INSTITUTO DE EDUCACION TECN...,611311,Escuelas de educación superior del sector privado,4.216867,18.806432,18.393782,0.000000,...,0.000000,0.052165,3.076121,26.359833,6.465068,14.077164,POINT (633294.5816255995 2129049.1244717026),0.0,Rain,0.0
182,Colima,0318,007,USAER ITINERANTE,611182,Escuelas del sector público de educación para ...,6.632653,20.629159,19.384615,0.000000,...,0.000000,2.263374,3.086420,23.971193,7.929969,12.397119,POINT (634564.0716057688 2129589.2442548405),0.0,Rain,0.0
183,Colima,0712,024,VILANOVA COLEGIO 06PJN0049Y PREESCOLAR,611172,Escuelas del sector público que combinan diver...,3.418803,21.231918,19.435484,0.000000,...,0.000000,0.332640,3.783784,23.991684,9.147609,15.634096,POINT (634214.992248468 2128619.76881661),0.0,Rain,0.0


In [25]:
i["asa"] = 1

In [26]:
i

localidad                                                Colima
ageb                                                       0318
manzana                                                     052
nom_estab                             UNIVERSIDAD UNIVER COLIMA
codigo_act                                               611311
nombre_act    Escuelas de educación superior del sector privado
P6A14NAE                                               6.632653
SBASC                                                 20.629159
PSDSS                                                 19.384615
OVSDE                                                       0.0
OVSEE                                                  0.102881
OVSAE                                                       0.0
OVPT                                                   2.263374
OVSREF                                                  3.08642
OVSINT                                                23.971193
OVSCEL                                  

In [33]:
df_full = pd.DataFrame()
for f in asc_files:
  df_full = work_progress(f, gdf, df_full)

In [14]:
asc_files[:2]

['data/Rasters\\Hydrology\\Rain____0.0.asc',
 'data/Rasters\\Hydrology\\Rain____10200.0.asc']

In [21]:
df_full.to_csv('data/denue_inegi_final.csv', index=False)

In [34]:
df_full

,localidad,ageb,manzana,nom_estab,codigo_act,nombre_act,P6A14NAE,SBASC,PSDSS,OVSDE,...,OVSAE,OVPT,OVSREF,OVSINT,OVSCEL,OVHAC,geometry,v_asc,variable,step
0,Colima,0727,031,FARMACIA INTERCAMBIABLE,464111,Farmacias sin minisúper,2.805611,25.378627,20.178042,0.000000,...,0.068744,1.489459,4.147571,27.978918,6.645280,15.879927,POINT (633121.9573959348 2128794.7203419697),NaN,Rain,0.0
1,Colima,0746,034,FARMACIA GUADALUPANA,464112,Farmacias con minisúper,1.923077,10.915493,20.382166,0.109529,...,0.000000,0.109529,2.957284,18.181818,6.681271,8.433735,POINT (633791.8220921274 2128170.2879505726),0.00000,Rain,0.0
2,Colima,0712,006,FARMACIA NUEVA IMPERIAL,464111,Farmacias sin minisúper,3.418803,21.231918,19.435484,0.000000,...,0.000000,0.332640,3.783784,23.991684,9.147609,15.634096,POINT (634823.7113330765 2128769.6261537718),0.00000,Rain,0.0
3,Colima,0750,003,FARMACIA MI BARRIO,464111,Farmacias sin minisúper,4.090909,24.190178,20.730634,0.846325,...,0.668151,0.846325,4.276169,31.403118,8.685969,14.610245,POINT (633641.7836969274 2128038.4058942627),NaN,Rain,0.0
4,Colima,0750,010,FARMACIA LA PROVIDENCIA,464111,Farmacias sin minisúper,4.090909,24.190178,20.730634,0.846325,...,0.668151,0.846325,4.276169,31.403118,8.685969,14.610245,POINT (633457.0221438431 2127859.2699528662),NaN,Rain,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44950,Colima,0731,026,UNIVERSIDAD MULTITECNICA PROFESIONAL,611311,Escuelas de educación superior del sector privado,4.306220,15.492958,17.206133,0.255537,...,0.511073,0.724020,1.618399,15.459966,5.749574,5.749574,POINT (632827.9149384073 2128394.2765015755),NaN,Velocity,9900.0
44951,Colima,1265,020,UNIVERSIDAD INETEP INSTITUTO DE EDUCACION TECN...,611311,Escuelas de educación superior del sector privado,4.216867,18.806432,18.393782,0.000000,...,0.000000,0.052165,3.076121,26.359833,6.465068,14.077164,POINT (633294.5816255995 2129049.1244717026),1.17633,Velocity,9900.0
44952,Colima,0318,007,USAER ITINERANTE,611182,Escuelas del sector público de educación para ...,6.632653,20.629159,19.384615,0.000000,...,0.000000,2.263374,3.086420,23.971193,7.929969,12.397119,POINT (634564.0716057688 2129589.2442548405),NaN,Velocity,9900.0
44953,Colima,0712,024,VILANOVA COLEGIO 06PJN0049Y PREESCOLAR,611172,Escuelas del sector público que combinan diver...,3.418803,21.231918,19.435484,0.000000,...,0.000000,0.332640,3.783784,23.991684,9.147609,15.634096,POINT (634214.992248468 2128619.76881661),NaN,Velocity,9900.0
